In [55]:
from lxml import html
import requests
import shutil
import os


page1_url = "https://web.archive.org/web/20160310043011/http://tahoemuseum.org/"
page2_url = "https://web.archive.org/web/20160320132803/http://tahoemuseum.org/page/2/"

In [56]:
url = page1_url
page = requests.get(url)
xml_tree = html.fromstring(page.content)


# Xpaths
def get_post_from_content(xml_tree, post_num):
    post = post_num
    post_xpath = f'//*[@id="content"]/article[{post}]'
    title_xpath = f'{post_xpath}/div/header/div/div/h2/a'
    image_xpath = f'{post_xpath}/div/div/a/img'

    title = xml_tree.xpath(title_xpath)[0].attrib['title']
    image = xml_tree.xpath(image_xpath)[0]
    image_url = image.attrib['src']
    
    return title, image_url


all_posts_xpath = f'//*[@id="content"]/article'
all_posts = xml_tree.xpath(all_posts_xpath)
post_count = len(all_posts)

for post_num in range(1, post_count+1):
    title, image_url = get_post_from_content(xml_tree, post_num)
    dirname = make_directory(title)
    download_image(image_url, dirname)

In [50]:
def download_image(image_url, folder_name):
    image_extension = '.' + image_url.split('.')[-1]
    filename = image_url.split('/')[-1]
    response = requests.get(image_url, stream=True)
    with open(f'./{folder_name}/{filename}','wb') as out_file:
        shutil.copyfileobj(response.raw, out_file)
    del response

def make_directory(title):
    filtered_title = "".join(list(filter(
        lambda c: c.isalpha() or c==' ', title)))
    filtered_title = filtered_title.replace(' ','_')
    try:
        os.mkdir(filtered_title)
    except FileExistsError:
        pass
    return filtered_title

In [53]:
for post_num in [1]:
    title, image_url = get_post_from_content(xml_tree, post_num)
    dirname = make_directory(title)
    download_image(image_url, dirname)